In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
pip install google-adk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.9/319.9 kB 4.4 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.0
    Uninstalling protobuf-6.33.0:
      Successfully uninstalled protobuf-6.33.0
  Attempting uninstall: cachetools
    Found existing installation: cachetools 6.2.1
    Uninstalling cachetools-6.2.1:
      Successfully uninstalled cachetools-6.2.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
google-cloud-translate 3.12.1 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.19.5, but you have protobuf 5.29.5 which is incompatible.
ray 2.51.1 requires click!=8.3.0,>=7.0, but you have click 8.3.0 which is incompatible.
bigframes 2.12.0 requires ric

In [4]:
import os
from kaggle_secrets import UserSecretsClient

try:
    GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
    print("✅ Setup and authentication complete.")
except Exception as e:
    print(
        f"🔑 Authentication Error: Please make sure you have added 'GOOGLE_API_KEY' to your Kaggle secrets. Details: {e}"
    )

✅ Setup and authentication complete.


In [5]:
from google.genai import types

from google.adk.agents import LlmAgent
from google.adk.models.google_llm import Gemini
from google.adk.runners import InMemoryRunner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search, AgentTool, ToolContext
from google.adk.code_executors import BuiltInCodeExecutor

print("✅ ADK components imported successfully.")

✅ ADK components imported successfully.


In [6]:
def show_python_code_and_result(response):
    for i in range(len(response)):
        # Check if the response contains a valid function call result from the code executor
        if (
            (response[i].content.parts)
            and (response[i].content.parts[0])
            and (response[i].content.parts[0].function_response)
            and (response[i].content.parts[0].function_response.response)
        ):
            response_code = response[i].content.parts[0].function_response.response
            if "result" in response_code and response_code["result"] != "```":
                if "tool_code" in response_code["result"]:
                    print(
                        "Generated Python Code >> ",
                        response_code["result"].replace("tool_code", ""),
                    )
                else:
                    print("Generated Python Response >> ", response_code["result"])


print("✅ Helper functions defined.")

✅ Helper functions defined.


In [7]:
retry_config = types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504],  # Retry on these HTTP errors
)

In [8]:
# Pay attention to the docstring, type hints, and return value.
def get_fee_for_payment_method(method: str) -> dict:
    """Looks up the transaction fee percentage for a given payment method.

    This tool simulates looking up a company's internal fee structure based on
    the name of the payment method provided by the user.

    Args:
        method: The name of the payment method. It should be descriptive,
                e.g., "platinum credit card" or "bank transfer".

    Returns:
        Dictionary with status and fee information.
        Success: {"status": "success", "fee_percentage": 0.02}
        Error: {"status": "error", "error_message": "Payment method not found"}
    """
    # This simulates looking up a company's internal fee structure.
    fee_database = {
        "platinum credit card": 0.02,  # 2%
        "gold debit card": 0.035,  # 3.5%
        "bank transfer": 0.01,  # 1%
    }

    fee = fee_database.get(method.lower())
    if fee is not None:
        return {"status": "success", "fee_percentage": fee}
    else:
        return {
            "status": "error",
            "error_message": f"Payment method '{method}' not found",
        }


print("✅ Fee lookup function created")
print(f"💳 Test: {get_fee_for_payment_method('platinum credit card')}")

✅ Fee lookup function created
💳 Test: {'status': 'success', 'fee_percentage': 0.02}


In [9]:
def get_exchange_rate(base_currency: str, target_currency: str) -> dict:
    """Looks up and returns the exchange rate between two currencies.

    Args:
        base_currency: The ISO 4217 currency code of the currency you
                       are converting from (e.g., "USD").
        target_currency: The ISO 4217 currency code of the currency you
                         are converting to (e.g., "EUR").

    Returns:
        Dictionary with status and rate information.
        Success: {"status": "success", "rate": 0.93}
        Error: {"status": "error", "error_message": "Unsupported currency pair"}
    """

    # Static data simulating a live exchange rate API
    # In production, this would call something like: requests.get("api.exchangerates.com")
    rate_database = {
        "usd": {
            "eur": 0.93,  # Euro
            "jpy": 157.50,  # Japanese Yen
            "inr": 83.58,  # Indian Rupee
        }
    }

    # Input validation and processing
    base = base_currency.lower()
    target = target_currency.lower()

    # Return structured result with status
    rate = rate_database.get(base, {}).get(target)
    if rate is not None:
        return {"status": "success", "rate": rate}
    else:
        return {
            "status": "error",
            "error_message": f"Unsupported currency pair: {base_currency}/{target_currency}",
        }


print("✅ Exchange rate function created")
print(f"💱 Test: {get_exchange_rate('USD', 'EUR')}")

✅ Exchange rate function created
💱 Test: {'status': 'success', 'rate': 0.93}


In [10]:
# Currency agent with custom function tools
currency_agent = LlmAgent(
    name="currency_agent",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    instruction="""You are a smart currency conversion assistant.

    For currency conversion requests:
    1. Use `get_fee_for_payment_method()` to find transaction fees
    2. Use `get_exchange_rate()` to get currency conversion rates
    3. Check the "status" field in each tool's response for errors
    4. Calculate the final amount after fees based on the output from `get_fee_for_payment_method` and `get_exchange_rate` methods and provide a clear breakdown.
    5. First, state the final converted amount.
        Then, explain how you got that result by showing the intermediate amounts. Your explanation must include: the fee percentage and its
        value in the original currency, the amount remaining after the fee, and the exchange rate used for the final conversion.

    If any tool returns status "error", explain the issue to the user clearly.
    """,
    tools=[get_fee_for_payment_method, get_exchange_rate],
)

print("✅ Currency agent created with custom function tools")
print("🔧 Available tools:")
print("  • get_fee_for_payment_method - Looks up company fee structure")
print("  • get_exchange_rate - Gets current exchange rates")

✅ Currency agent created with custom function tools
🔧 Available tools:
  • get_fee_for_payment_method - Looks up company fee structure
  • get_exchange_rate - Gets current exchange rates


In [11]:
# Test the currency agent
currency_runner = InMemoryRunner(agent=currency_agent)
_ = await currency_runner.run_debug(
    "I want to convert 500 US Dollars to Euros using my Platinum Credit Card. How much will I receive?"
)


 ### Created new session: debug_session_id

User > I want to convert 500 US Dollars to Euros using my Platinum Credit Card. How much will I receive?


currency_agent > You will receive 455.70 Euros.

Here's the breakdown:
1. The transaction fee for using a platinum credit card is 2%.
2. The fee amount is 10.00 USD (2% of 500 USD).
3. The remaining amount is 490.00 USD (500 USD - 10.00 USD).
4. The exchange rate from USD to EUR is 0.93.
5. The final amount in EUR is 455.70 EUR (490.00 USD * 0.93).


In [12]:
calculation_agent = LlmAgent(
    name="CalculationAgent",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    instruction="""You are a specialized calculator that ONLY responds with Python code. You are forbidden from providing any text, explanations, or conversational responses.
 
     Your task is to take a request for a calculation and translate it into a single block of Python code that calculates the answer.
     
     **RULES:**
    1.  Your output MUST be ONLY a Python code block.
    2.  Do NOT write any text before or after the code block.
    3.  The Python code MUST calculate the result.
    4.  The Python code MUST print the final result to stdout.
    5.  You are PROHIBITED from performing the calculation yourself. Your only job is to generate the code that will perform the calculation.
   
    Failure to follow these rules will result in an error.
       """,
    code_executor=BuiltInCodeExecutor(),  # Use the built-in Code Executor Tool. This gives the agent code execution capabilities
)

In [13]:
enhanced_currency_agent = LlmAgent(
    name="enhanced_currency_agent",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    # Updated instruction
    instruction="""You are a smart currency conversion assistant. You must strictly follow these steps and use the available tools.

  For any currency conversion request:

   1. Get Transaction Fee: Use the get_fee_for_payment_method() tool to determine the transaction fee.
   2. Get Exchange Rate: Use the get_exchange_rate() tool to get the currency conversion rate.
   3. Error Check: After each tool call, you must check the "status" field in the response. If the status is "error", you must stop and clearly explain the issue to the user.
   4. Calculate Final Amount (CRITICAL): You are strictly prohibited from performing any arithmetic calculations yourself. You must use the calculation_agent tool to generate Python code that calculates the final converted amount. This 
      code will use the fee information from step 1 and the exchange rate from step 2.
   5. Provide Detailed Breakdown: In your summary, you must:
       * State the final converted amount.
       * Explain how the result was calculated, including:
           * The fee percentage and the fee amount in the original currency.
           * The amount remaining after deducting the fee.
           * The exchange rate applied.
    """,
    tools=[
        get_fee_for_payment_method,
        get_exchange_rate,
        AgentTool(agent=calculation_agent),  # Using another agent as a tool!
    ],
)

print("✅ Enhanced currency agent created")
print("🎯 New capability: Delegates calculations to specialist agent")
print("🔧 Tool types used:")
print("  • Function Tools (fees, rates)")
print("  • Agent Tool (calculation specialist)")

✅ Enhanced currency agent created
🎯 New capability: Delegates calculations to specialist agent
🔧 Tool types used:
  • Function Tools (fees, rates)
  • Agent Tool (calculation specialist)


In [14]:
# Define a runner
enhanced_runner = InMemoryRunner(agent=enhanced_currency_agent)

In [15]:
response = await enhanced_runner.run_debug(
    "Convert 1,250 USD to INR using a Bank Transfer. Show me the precise calculation."
)


 ### Created new session: debug_session_id

User > Convert 1,250 USD to INR using a Bank Transfer. Show me the precise calculation.


enhanced_currency_agent > The conversion of 1,250 USD to INR results in 103,430.25 INR.

Here's the breakdown of the calculation:
*   The transaction fee for a Bank Transfer is 1.00%, which amounts to 12.50 USD.
*   After deducting the fee, the remaining amount is 1,237.50 USD.
*   This amount was converted using an exchange rate of 1 USD = 83.58 INR.
*   The final converted amount is 103,430.25 INR.


In [16]:
show_python_code_and_result(response)

Generated Python Response >>  The previous request has been processed. The fee percentage is 1.00%, the fee amount is 12.50 USD, the amount after fee is 1237.50 USD, and the converted amount is 103430.25 INR with an exchange rate of 83.58. No more outputs are needed.


Day2b

In [17]:
import uuid
from google.genai import types

from google.adk.agents import LlmAgent
from google.adk.models.google_llm import Gemini
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService

from google.adk.tools.mcp_tool.mcp_toolset import McpToolset
from google.adk.tools.tool_context import ToolContext
from google.adk.tools.mcp_tool.mcp_session_manager import StdioConnectionParams
from mcp import StdioServerParameters

from google.adk.apps.app import App, ResumabilityConfig
from google.adk.tools.function_tool import FunctionTool

print("✅ ADK components imported successfully.")

✅ ADK components imported successfully.


In [18]:
retry_config=types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504] # Retry on these HTTP errors
)

In [19]:
# MCP integration with Everything Server
mcp_image_server = McpToolset(
    connection_params=StdioConnectionParams(
        server_params=StdioServerParameters(
            command='npx',  # Run MCP server via npx
            args=["-y",  # Argument for npx to auto-confirm install
                  "@modelcontextprotocol/server-everything",
                ],
            tool_filter=['getTinyImage']
        ),
        timeout=30,
    )
)

print("✅ MCP Tool created")

✅ MCP Tool created


In [20]:
# Create image agent with MCP integration
image_agent = LlmAgent(
   model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
   ),
   name='image_agent',
   instruction='Use the MCP Tool to generate images for user queries',
   tools = [mcp_image_server]
)

In [21]:
from google.adk.runners import InMemoryRunner

runner = InMemoryRunner(agent=image_agent)

In [22]:
response = await runner.run_debug("Provide a sample tiny image", verbose=True)


 ### Created new session: debug_session_id

User > Provide a sample tiny image


/usr/local/lib/python3.11/dist-packages/google/adk/tools/mcp_tool/mcp_tool.py:101: UserWarning: [EXPERIMENTAL] BaseAuthenticatedTool: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  super().__init__(


image_agent > [Calling tool: getTinyImage({})]
image_agent > [Tool result: {'content': [{'type': 'text', 'text': 'This is a tiny image:'}, {'type': 'image', 'data': 'iVBORw0KG...]
image_agent > This is a tiny image:



In [23]:
from IPython.display import display, Image as IPImage
import base64

for event in response:
  if event.content and event.content.parts:
      for part in event.content.parts:
          if hasattr(part, 'function_response') and part.function_response:
              for item in part.function_response.response.get('content', []):
                  if item.get('type') == 'image':
                      display(IPImage(data=base64.b64decode(item['data'])))

In [24]:
McpToolset(
    connection_params=StdioConnectionParams(
        server_params=StdioServerParameters(
            command='npx',
            args=[
                '-y',
                'mcp-remote',
                'https://www.kaggle.com/mcp'
            ],
        ),
        timeout=30,
    )
)

In [27]:
McpToolset(
    connection_params=StreamableHTTPServerParams(
        url="https://api.githubcopilot.com/mcp/",
        headers={
            "Authorization": f"Bearer {GITHUB_TOKEN}",
            "X-MCP-Toolsets": "all",
            "X-MCP-Readonly": "true"
        },
    ),
)

NameError: name 'StreamableHTTPServerParams' is not defined

In [28]:
LARGE_ORDER_THRESHOLD = 5

def place_shipping_order(num_containers: int, destination: str, tool_context: ToolContext) -> dict:
  """Places a shipping order. Requires approval if ordering more than 5 containers (LARGE_ORDER_THRESHOLD).
  
  Args:
      num_containers: Number of containers to ship
      destination: Shipping destination
  
  Returns:
      Dictionary with order status
  """

  # -----------------------------------------------------------------------------------------------
  # -----------------------------------------------------------------------------------------------
  # SCENARIO 1: Small orders (≤5 containers) auto-approve
  if num_containers <= LARGE_ORDER_THRESHOLD:
      return {
          "status": "approved",
          "order_id": f"ORD-{num_containers}-AUTO",
          "num_containers": num_containers,
          "destination": destination,
          "message": f"Order auto-approved: {num_containers} containers to {destination}"
      }

  # -----------------------------------------------------------------------------------------------
  # -----------------------------------------------------------------------------------------------   
  # SCENARIO 2: This is the first time this tool is called. Large orders need human approval - PAUSE here.
  if not tool_context.tool_confirmation:
      tool_context.request_confirmation(
          hint=f"⚠️ Large order: {num_containers} containers to {destination}. Do you want to approve?",
          payload={"num_containers": num_containers, "destination": destination}
      )
      return { # This is sent to the Agent
          "status": "pending",
          "message": f"Order for {num_containers} containers requires approval"
      }

  # -----------------------------------------------------------------------------------------------
  # -----------------------------------------------------------------------------------------------   
  # SCENARIO 3: The tool is called AGAIN and is now resuming. Handle approval response - RESUME here.
  if tool_context.tool_confirmation.confirmed:
      return {
          "status": "approved",
          "order_id": f"ORD-{num_containers}-HUMAN",
          "num_containers": num_containers,
          "destination": destination,
          "message": f"Order approved: {num_containers} containers to {destination}"
      }
  else:
      return {
          "status": "rejected",
          "message": f"Order rejected: {num_containers} containers to {destination}"
      }

print("✅ Long-running functions created!")

✅ Long-running functions created!


In [29]:
# Create shipping agent with pausable tool
shipping_agent = LlmAgent(
  name="shipping_agent",
  model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
  ),
  instruction="""You are a shipping coordinator assistant.
  
  When users request to ship containers:
   1. Use the place_shipping_order tool with the number of containers and destination
   2. If the order status is 'pending', inform the user that approval is required
   3. After receiving the final result, provide a clear summary including:
      - Order status (approved/rejected)
      - Order ID (if available)
      - Number of containers and destination
   4. Keep responses concise but informative
  """,
  tools=[
      FunctionTool(func=place_shipping_order)
  ]
)

print("✅ Shipping Agent created!")

✅ Shipping Agent created!


In [30]:
# Wrap the agent in a resumable app - THIS IS THE KEY FOR LONG-RUNNING OPERATIONS!
shipping_app = App(
  name="shipping_coordinator",
  root_agent=shipping_agent,
  resumability_config=ResumabilityConfig(is_resumable=True)
)

print("✅ Resumable app created!")

✅ Resumable app created!


/tmp/ipykernel_48/598580593.py:5: UserWarning: [EXPERIMENTAL] ResumabilityConfig: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  resumability_config=ResumabilityConfig(is_resumable=True)


In [31]:
session_service = InMemorySessionService()

# Create runner with the resumable app
shipping_runner = Runner(
    app=shipping_app,  # Pass the app instead of the agent
    session_service=session_service
)

print("✅ Runner created!")

✅ Runner created!


Section 4: Building the Workflow

In [32]:
def check_for_approval(events):
  """Check if events contain an approval request.
  
  Returns:
      dict with approval details or None
  """
  for event in events:
      if event.content and event.content.parts:
          for part in event.content.parts:
              if part.function_call and part.function_call.name == "adk_request_confirmation":
                  return {
                      'approval_id': part.function_call.id,
                      'invocation_id': event.invocation_id
                  }
  return None

In [33]:
def print_agent_response(events):
  """Print agent's text responses from events."""
  for event in events:
      if event.content and event.content.parts:
          for part in event.content.parts:
              if part.text:
                  print(f"Agent > {part.text}")

In [34]:
def create_approval_response(approval_info, approved):
  """Create approval response message."""
  confirmation_response = types.FunctionResponse(
      id=approval_info['approval_id'],
      name="adk_request_confirmation",
      response={"confirmed": approved}
  )
  return types.Content(role="user", parts=[types.Part(function_response=confirmation_response)])

print("✅ Helper functions defined")

✅ Helper functions defined


In [35]:
async def run_shipping_workflow(query: str, auto_approve: bool = True):
  """Runs a shipping workflow with approval handling.
  
  Args:
      query: User's shipping request
      auto_approve: Whether to auto-approve large orders (simulates human decision)
  """

  print(f"\n{'='*60}")
  print(f"User > {query}\n")

  # Generate unique session ID
  session_id = f"order_{uuid.uuid4().hex[:8]}"

  # Create session
  await session_service.create_session(
      app_name="shipping_coordinator",
      user_id="test_user",
      session_id=session_id
  )

  query_content = types.Content(role="user", parts=[types.Part(text=query)])
  events = []
    

  # -----------------------------------------------------------------------------------------------
  # -----------------------------------------------------------------------------------------------  
  # STEP 1: Send initial request to the Agent. If num_containers > 5, the Agent returns the special `adk_request_confirmation` event
  async for event in shipping_runner.run_async(
      user_id="test_user",
      session_id=session_id,
      new_message=query_content
  ):
      events.append(event)
      

  # -----------------------------------------------------------------------------------------------
  # -----------------------------------------------------------------------------------------------  
  # STEP 2: Loop through all the events generated and check if `adk_request_confirmation` is present.
  approval_info = check_for_approval(events)
    

  # -----------------------------------------------------------------------------------------------
  # -----------------------------------------------------------------------------------------------  
  # STEP 3: If the event is present, it's a large order - HANDLE APPROVAL WORKFLOW
  if approval_info:
      print(f"⏸️  Pausing for approval...")
      print(f"🤔 Human Decision: {'APPROVE ✅' if auto_approve else 'REJECT ❌'}\n")

      # PATH A: Resume the agent by calling run_async() again with the approval decision
      async for event in shipping_runner.run_async(
          user_id="test_user",
          session_id=session_id,
          new_message=create_approval_response(approval_info, auto_approve), # Send human decision here
          invocation_id=approval_info['invocation_id']  # Critical: same invocation_id tells ADK to RESUME
      ):
          if event.content and event.content.parts:
              for part in event.content.parts:
                  if part.text:
                      print(f"Agent > {part.text}")

  # -----------------------------------------------------------------------------------------------
  # -----------------------------------------------------------------------------------------------  
  else:
      # PATH B: If the `adk_request_confirmation` is not present - no approval needed - order completed immediately.
      print_agent_response(events)

  print(f"{'='*60}\n")

print("✅ Workflow function ready")

✅ Workflow function ready


In [36]:
# Demo 1: It's a small order. Agent receives auto-approved status from tool
await run_shipping_workflow("Ship 3 containers to Singapore")
 
# Demo 2: Workflow simulates human decision: APPROVE ✅
await run_shipping_workflow("Ship 10 containers to Rotterdam", auto_approve=True)

# Demo 3: Workflow simulates human decision: REJECT ❌
await run_shipping_workflow("Ship 8 containers to Los Angeles", auto_approve=False)


User > Ship 3 containers to Singapore



Agent > Shipping order approved!
Order ID: ORD-3-AUTO
Destination: Singapore
Number of containers: 3


User > Ship 10 containers to Rotterdam



/usr/local/lib/python3.11/dist-packages/google/adk/tools/tool_context.py:92: UserWarning: [EXPERIMENTAL] ToolConfirmation: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  ToolConfirmation(
/usr/local/lib/python3.11/dist-packages/google/adk/agents/invocation_context.py:298: UserWarning: [EXPERIMENTAL] BaseAgentState: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  self.agent_states[event.author] = BaseAgentState()


⏸️  Pausing for approval...
🤔 Human Decision: APPROVE ✅

Agent > Shipping order approved. Order ID: ORD-10-HUMAN. 10 containers will be shipped to Rotterdam.


User > Ship 8 containers to Los Angeles



⏸️  Pausing for approval...
🤔 Human Decision: REJECT ❌

Agent > Order rejected: 8 containers to Los Angeles.

